# ClassifyWD_dep

# - Schwellwert - live - chatnor+targer - end score ->  0.55 -> 0.58
- Responses chatnoir - local gespeichert - hier end score -> 0.55 -> 0.45

In [1]:
import requests
import xml.etree.ElementTree as ET
import time
from collections import Counter
from itertools import chain
"""Approximately simulates trec_eval using pytrec_eval."""

import argparse
import os
import sys
from statistics import mean 
import pytrec_eval

import copy
import pandas as pd

import requests
import json
import bs4
import regex as re
import time


def read_results(filename):
    
    with open(filename) as f:
        content = f.readlines()

    content= [x.strip() for x in content]
    i=1
    idx=0
    topics ={}
    while idx <751:
        topics[i] = content[idx:idx+15]
        idx=idx+15
        i=i+1
    return topics
def remove_htmlTags(snippet):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', snippet)

In [2]:
filename="files_local/output_title_snippet.run"
results= read_results(filename)
#REDUKTION RESPONSES AUS TXT
topics_responses={}
for topic_id in range(1,51):
    def responses(results, topic_id):
        m_responses = [(row.split("\t")[2],row.split("\t")[4], row.split("\t")[6], row.split("\t")[7]) for row in results[topic_id]]
        m_liste=[ {'trec_id': trec_id, 'score': float(score), 'title': remove_htmlTags(title), 'snippet':remove_htmlTags(snippet)}   for trec_id, score, title, snippet in m_responses]          
        resp={'results': m_liste}
        return resp
    response = responses(results, topic_id)
    topics_responses[topic_id] = response

In [3]:

'''
def calculateDCG(args_run):
   
    args_qrel='files_local/touche2020-task2-relevance-withbaseline.qrel'
    args_measure='ndcg_cut_5'
    with open(args_qrel, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(args_run, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)
    
    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, {args_measure})

    results = evaluator.evaluate(run)
    
    def print_line(measure, scope, value):
        #scope = query_id = topic_id
        print('{:25s}{:8s}{:.22f}'.format(measure, scope, value))
    avg_DCG = []
    for query_id, query_measures in results.items():
   
        for measure, value in sorted(query_measures.items()):
            avg_DCG.append(value)
            #print_line(measure, query_id, value)
    #print(mean(avg_DCG))
    return mean(avg_DCG)
'''

def nDCG(targer_modell,score_string):
    args_run='files_local/targer_output_ntopics_50_15_False_False_False_'+str(targer_model)+'_underscore_'+score_string+'.run'
    result = calculateDCG(args_run)
    return result

In [4]:
def isArgument(topic_id):
    not_args_list= [6,13]
    if topic_id not in not_args_list:
        return True
    else:
        return False
def response_targer_api(doc, targer_model):
    #ex: targer = classifyWD, classifyNewWD, classifyWD_dep

    import regex as re
    payload=doc
    url = "https://demo.webis.de/targer-api/"+targer_model    
    headers = {
        'Content-Type': 'text/plain'
    }
    def targer():
        try:
            response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8'))
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError:
            time.sleep(1)
            return targer()
    response = targer()
    return response
def get_argument_score(resp_as_list, targer_model, underscore):
    #
    if targer_model!="classifyNewWD":
        count_arg_labels=0
        sum_probs=0.0
        arg_labels_probas=[]
        for ents_list in resp_as_list:
            for ent in ents_list:
                if ent['label'].endswith("-B") or ent['label'].endswith("-I"):
                    count_arg_labels += 1
                    sum_probs = sum_probs + float(ent['prob'])
                    arg_labels_probas.append((ent['label'],ent['prob']))
        if count_arg_labels==0:
            return 0
        else:
            avg_argScore = sum_probs/count_arg_labels
            if avg_argScore<=float(underscore):
                return 0
            else:
                return avg_argScore #arg_labels_probas
    else:
        count_arg_labels=0
        for ents_list in resp_as_list:
            for ent in ents_list:
                if ent['label'].endswith("-B") or ent['label'].endswith("-I"):
                    count_arg_labels += 1
        avg_argScore = count_arg_labels/len(resp_as_list)
        if avg_argScore<=float(underscore):
            return 0
        else:
            return avg_argScore #arg_labels_probas
def add_arg_score_to_response(ids_argscores,resp):
    new_resp=[]
    for doc in resp['results']:
        actual_id = doc['trec_id']
        rel_score=float(doc['score'])
        agr_score= ids_argscores[actual_id]
        #change end_score
        doc['score'] = rel_score*agr_score #rel_score*(1+agr_score)
        new_resp.append(doc)
    return {'results':sorted(new_resp,key= lambda doc: doc['score'], reverse=True)} #response with new sorting by new score

In [5]:
targer_model="classifyWD_dep"
topics_argument_labels={}
for topic_id, response in topics_responses.items():
    if isArgument(topic_id)==True:
        ids_argscores={}
        #get argument scores for all documents in the result.
        targer_doc={}
        for e in response['results']:
            doc= e['title']+ ". " + e['snippet']
            trec_id=e['trec_id']
            doc_labels = response_targer_api(doc, targer_model)
            targer_doc[trec_id]=doc_labels
        topics_argument_labels[topic_id]=targer_doc

In [6]:
topics_argument_labels[1]

{'clueweb12-1214wb-88-29751': [[{'label': 'O',
    'prob': '0.7817677',
    'token': 'Difference'},
   {'label': 'O', 'prob': '0.83270216', 'token': 'Between'},
   {'label': 'O', 'prob': '0.802422', 'token': 'Love'},
   {'label': 'O', 'prob': '0.8660865', 'token': 'And'},
   {'label': 'O', 'prob': '0.83912605', 'token': 'Sex'},
   {'label': 'O', 'prob': '0.9082738', 'token': '-'},
   {'label': 'O', 'prob': '0.9184673', 'token': 'Student.com'},
   {'label': 'O', 'prob': '0.9302571', 'token': 'Articles'},
   {'label': 'O', 'prob': '0.89412856', 'token': '.'}],
  [{'label': 'O', 'prob': '0.2796378', 'token': 'Sex'},
   {'label': 'P-I', 'prob': '0.37010884', 'token': 'may'},
   {'label': 'P-I', 'prob': '0.42882606', 'token': 'or'},
   {'label': 'P-I', 'prob': '0.45782527', 'token': 'may'},
   {'label': 'P-I', 'prob': '0.48444054', 'token': 'not'},
   {'label': 'P-I', 'prob': '0.510208', 'token': 'include'},
   {'label': 'P-I', 'prob': '0.553338', 'token': 'penetration'},
   {'label': 'P-I'

In [7]:
underscore=0.0
scores=[]
ntopics_score=[]
while underscore<=0.7:
    n_arg_docs=0
    n_docs=[]
    scores.append(underscore)
    for topic_id, response in topics_responses.items():
        if topic_id not in [6,13]:
            all_labels=topics_argument_labels[topic_id]
            for trec_id, labels in all_labels.items():
                arg_score=get_argument_score(labels, targer_model, underscore)
                n_docs.append(arg_score)
                if arg_score!=0:
                    n_arg_docs=n_arg_docs+1
    print(len(n_docs))
    ntopics_score.append(n_arg_docs)
    underscore=round(underscore+0.05,2)
df= pd.DataFrame()
df['underscore']= scores
df['ndocs'] = ntopics_score
#df['proba'] =probas_score
#df=df.sort_values('nDCG5', ascending=False)

720
720
720
720
720
720
720
720
720
720
720
720
720
720
720


In [8]:
'''
es gibt dann 283 größer als 0
'''
df

,underscore,ndocs
0,0.00,283
1,0.05,283
2,0.10,283
3,0.15,283
4,0.20,283
5,0.25,283
6,0.30,281
7,0.35,278
8,0.40,272
9,0.45,229


In [9]:
def update_results_topic(topic_id, targer_model, underscore, response):
    ids_argscores={}
    #get argument scores for all documents in the result.
    for e in response['results']:
        doc= e['title']+ ". " + e['snippet']
        trec_id=e['trec_id']
        #labels_targer = response_targer_api(doc, targer_model)
        labels_targer=topics_argument_labels[topic_id][trec_id]
        argscore = get_argument_score(labels_targer, targer_model, underscore)
        ids_argscores[trec_id]=argscore
   # print(list(ids_argscores.values()))
    #update scores of all documents by adding argument scores
    end_response=add_arg_score_to_response(ids_argscores,response)
    return ids_argscores,end_response
def get_results(targer_model, underscore):
    score_string=str(underscore).replace('.','')
        #underscore=round(underscore,2)
    end_responses_all_topics={}
    all_argscores=[]
    for topic_id, response in topics_responses.items():
        end_response={}
        ids_argscore=dict()
       # print("=========="+ str(topic_id)+"===========")
        #print(response)
        if isArgument(topic_id)==True:
            tmp,end_response=update_results_topic(topic_id,targer_model, underscore, response)
           # print("test")
            ids_argscore=tmp
        else:
            print("Not argument query")
            end_response=response
            ids_argscore = ids_argscore
            print(ids_argscore)
        #print(end_response)
        end_responses_all_topics[topic_id] = end_response
        all_argscores.append(ids_argscore)

    out = open("files_local/targer_output_ntopics_50_15_False_False_False_"+str(targer_model)+"_underscore_"+score_string+".run", "w")

    answers=copy.deepcopy(end_responses_all_topics)
    for topic_id, end_results_pro_topic in answers.items():
        rank = 1
        for response in end_results_pro_topic['results']:
            buffer = topic_id, "Q0", response['trec_id'], rank, response['score'], "JackSparrowVanilla"
            #print(buffer)
            out.write(" ".join(map(str, buffer)) + "\n")
            rank += 1
    return all_argscores
def relationship():
    return True

In [10]:
def calculateDCG(args_run):
   
    args_qrel='files_local/touche2020-task2-relevance-withbaseline.qrel'
    args_measure='ndcg_cut_5'
    with open(args_qrel, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(args_run, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)
    
    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, {args_measure})

    results = evaluator.evaluate(run)
    
    ndcg_topic={}
    def print_line(measure, scope, value):
        #scope = query_id = topic_id
        print('{:25s}{:8s}{:.22f}'.format(measure, scope, value))
    avg_DCG = []
    for query_id, query_measures in results.items():
   
        for measure, value in sorted(query_measures.items()):
            avg_DCG.append(value)
            ndcg_topic[query_id] = value
            #print_line(measure, query_id, value)
    return ndcg_topic, mean(avg_DCG)
'''
def nDCG(targer_model, weights_as_string):
    args_run='files_local/output_ntopics_50_15_True_False_True_True_'+weights_as_string+'_max.run'
    result = calculateDCG(args_run)
    return result
'''

"\ndef nDCG(targer_model, weights_as_string):\n    args_run='files_local/output_ntopics_50_15_True_False_True_True_'+weights_as_string+'_max.run'\n    result = calculateDCG(args_run)\n    return result\n"

In [19]:
run compare_with_baseline.py files/touche2020-task2-relevance-withbaseline.qrel files_local/base_chatnoir.run files_local/expanded_output_ntopics_50_15_False_False_False_classifyWD_dep_underscore_055.run 'ndcg_cut_5'

 avg of chatnoir nDCG 0.570092
 avg of new aprroach nDCG 0.581630


# ndcg each topic between base chatnoir and argument approach

In [79]:
#run cal_pytrec_eval.py files_local/touche2020-task2-relevance-withbaseline.qrel files_local/expanded_output_ntopics_50_15_False_False_False_classifyWD_dep_underscore_055.run 'ndcg_cut_5'
run_file = "files_local/expanded_output_ntopics_50_15_False_False_False_classifyWD_dep_underscore_055.run"
ndcg_topic, avg_ndcg = calculateDCG(run_file)
base_ndcg_topic, base_avg_ndcg = calculateDCG('files_local/base_chatnoir.run')
print("===BASE====")
print(base_avg_ndcg)
print(base_ndcg_topic)

print("===ARG===")
print(avg_ndcg)
print(ndcg_topic)

===BASE====
0.5700917154792933
{'1': 0.7765732350040719, '2': 1.0, '3': 0.5296347172140421, '4': 0.6261806449294969, '5': 0.915209948681596, '6': 0.0, '7': 0.3930068676327362, '8': 0.0, '9': 0.3082168163143322, '10': 0.45860940638890707, '11': 0.7860137352654724, '12': 0.7883291114011227, '13': 0.7529334028221761, '14': 0.6608397947263839, '15': 0.3275546239420398, '16': 0.44436116952724447, '17': 0.4774770439198113, '18': 0.44615333764087994, '19': 0.830419897363192, '20': 0.16958010263680806, '21': 0.915209948681596, '22': 0.830419897363192, '23': 0.6608397947263839, '24': 0.6770117388019904, '25': 0.830419897363192, '26': 0.6164336326286644, '27': 0.7227265726449519, '28': 0.9269658250786468, '29': 0.5919819915518812, '30': 0.6548086577531307, '31': 0.9269658250786468, '32': 0.915209948681596, '33': 0.830419897363192, '34': 0.8930068676327362, '35': 1.0, '36': 0.4256571533334049, '37': 0.6875399693572165, '38': 1.0, '39': 0.3620634786734523, '40': 0.07816178655294431, '41': 0.159791

# average argument score each query topic
- underscore 0.0
- underscore 0.55

In [80]:
all_argscores=get_results(targer_model, 0.0)
i=1
arg_avgs_each_query={}
for e in all_argscores:
    if e ==dict():
        arg_avgs_each_query[i]=0.0
        i=i+1
    else:
        arg_avgs_each_query[i]=mean(list(e.values()))
        i=i+1
print(arg_avgs_each_query)

Not argument query
{}
Not argument query
{}
{1: 0.13130819201777777, 2: 0.3988413787411968, 3: 0.10211830352247035, 4: 0.22234191993198654, 5: 0.12416361844444444, 6: 0.0, 7: 0.22630707714583334, 8: 0.22564543224744563, 9: 0.20784490133165265, 10: 0.3095677847992311, 11: 0.2436447261320836, 12: 0.25818510002105266, 13: 0.0, 14: 0.4392248785666897, 15: 0.15808600089444447, 16: 0.08831583455555556, 17: 0.20719023272063494, 18: 0.061018848277777774, 19: 0.14655517644444443, 20: 0.1401636915592775, 21: 0.10574359282624855, 22: 0.03235805908333333, 23: 0.06194131261904762, 24: 0.04292250311111111, 25: 0.23049066593891296, 26: 0.09555689431077695, 27: 0.1563855379652635, 28: 0.06006380533333333, 29: 0.19218678454040405, 30: 0.17000260660753969, 31: 0.1996903110298686, 32: 0, 33: 0.2265274577410192, 34: 0.0652730490925926, 35: 0.28928941507905587, 36: 0.22443044934444448, 37: 0.29885079881159504, 38: 0.11808654278306878, 39: 0.20467929188333828, 40: 0.3126323104326649, 41: 0.29814166481818183

# relationship between arg scores and ndcg

In [81]:
'''
def sorted_dict_by_ndcg(m_dict):
    sorted_dict = dict(sorted(m_dict.items(), key=lambda kv: kv[1], reverse=True))
    return list(sorted_dict.keys()), sorted_dict
m_keys,base_ndcg_topic = sorted_dict_by_ndcg(ndcg_topic)
print(m_keys)
'''

'\ndef sorted_dict_by_ndcg(m_dict):\n    sorted_dict = dict(sorted(m_dict.items(), key=lambda kv: kv[1], reverse=True))\n    return list(sorted_dict.keys()), sorted_dict\nm_keys,base_ndcg_topic = sorted_dict_by_ndcg(ndcg_topic)\nprint(m_keys)\n'

In [83]:
import xml.etree.ElementTree as ET
import pandas as pd

n_topics=50

def get_titles(file):
    tree = ET.parse(file)
    root = tree.getroot()
    titles = []
    i=1
    for title in root.iter('title'):
        if i<=int(n_topics):
            title = title.text.strip()
            titles.append(title)
    return titles
get_titles("topics-task-2.xml")

['What is the difference between sex and love?',
 'Which is better, a laptop or a desktop?',
 'Which is better, Canon or Nikon?',
 'What are the best dish detergents?',
 'What are the best cities to live in?',
 'What is the longest river in the U.S.?',
 'Which is healthiest: coffee, green tea or black tea and why?',
 'What are the advantages and disadvantages of PHP over Python and vice versa?',
 'Why is Linux better than Windows?',
 'How to sleep better?',
 'Should I buy an LCD TV or a plasma TV?',
 'Train or plane? Which is the better choice?',
 'What is the highest mountain on Earth?',
 'Should one prefer Chinese medicine or Western medicine?',
 'What are the best washing machine brands?',
 'Should I buy or rent?',
 'Do you prefer cats or dogs, and why?',
 'What is the better way to grill outdoors: gas or charcoal?',
 'Which is better, MAC or PC?',
 'What is better: to use a brush or a sponge?',
 'Which is better, Linux or Microsoft?',
 'Which is better, Pepsi or Coke?',
 'What is b

In [91]:
import pandas as pd

In [99]:
arg_avgs_each_query['6'] = 0.0
arg_avgs_each_query['13'] = 0.0
len(arg_avgs_each_query.values())

50

In [94]:
len(base_ndcg_topic.values())
len(arg_avgs_each_query.values())

48

In [101]:
df=pd.DataFrame()
df['base'] = list(base_ndcg_topic.values())
df['arg'] = list(ndcg_topic.values())
df['avg_arg_score'] = arg_avgs_each_query.values()
df.avg_arg_score.corr(df.base - df.arg)

0.04676211082528207

In [102]:
    
def plot_comparing():
    from statistics import mean 
    from plotly import graph_objects as go
    import pandas as pd

    fig3 = go.Figure(
            data=[
                go.Scatter(
                    name="base approach",
                    x=list(base_ndcg_topic.keys()),
                    y=list(base_ndcg_topic.values()),
                    mode='lines+markers',
                    #offsetgroup=0,
                ),
                go.Scatter(
                    name="arg approach",
                    x=list(ndcg_topic.keys()),
                    y=list(ndcg_topic.values()),
                    mode='lines+markers',
                    #offsetgroup=1,
                ),
                go.Scatter(
                    name="avg arg score",
                    x=list(arg_avgs_each_query.keys()),
                    y=list(arg_avgs_each_query.values()),
                    mode='lines+markers',
                    #offsetgroup=1,
                ),
            ],
            layout=go.Layout(
                title="relationship between arguments and approaches",
                yaxis_title="ndcg/arg score"
            )
        )
    fig3.show()
    
    
    
    
    better_queries=[]
    for key, value in base_ndcg_topic.items():
        if ndcg_topic[key]>value:
            better_queries.append(key)
    print(better_queries)
    
    titles = get_titles("topics-task-2.xml")
    for query_id in better_queries:
        print(query_id+":\t" + titles[int(query_id)-1])
    
    
plot_comparing()

['3', '7', '8', '11', '14', '24', '25', '29', '33', '36', '39', '40', '43', '45']
3:	Which is better, Canon or Nikon?
7:	Which is healthiest: coffee, green tea or black tea and why?
8:	What are the advantages and disadvantages of PHP over Python and vice versa?
11:	Should I buy an LCD TV or a plasma TV?
14:	Should one prefer Chinese medicine or Western medicine?
24:	Which one is better, Netflix or Blockbuster?
25:	Which browser is better, Internet Explorer or Firefox?
29:	Which is better, Disneyland or Disney World?
33:	What is better: ASP or PHP?
36:	What IDE is better for Java: NetBeans or Eclipse?
39:	Is Java code more readable than code written in Scala?
40:	Which operating system has better performance: Windows 7 or Windows 8?
43:	Should I prefer a Leica camera over Nikon for portrait photographs?
45:	Which laptop has a better durability: HP or Dell?


# other score 0.55

In [85]:
all_argscores=get_results(targer_model, 0.55)
i=1
arg_avgs_each_query={}
for e in all_argscores:
    if e ==dict():
        arg_avgs_each_query[i]=0.0
        i=i+1
    else:
        arg_avgs_each_query[i]=mean(list(e.values()))
        i=i+1
print(arg_avgs_each_query)
plot_comparing()

Not argument query
{}
Not argument query
{}
{1: 0, 2: 0.24071837002095872, 3: 0.03777292446153846, 4: 0.038316767939393946, 5: 0, 6: 0.0, 7: 0.0367918958125, 8: 0.12690989607248335, 9: 0.07500700380784313, 10: 0.07646337726666667, 11: 0.07908229808446456, 12: 0, 13: 0.0, 14: 0.3170857691762135, 15: 0, 16: 0, 17: 0.03875329666666667, 18: 0, 19: 0, 20: 0.03687921820689655, 21: 0.07571677763577236, 22: 0, 23: 0, 24: 0.04292250311111111, 25: 0.04074758103703704, 26: 0, 27: 0.08600510198391151, 28: 0, 29: 0.1611407248737374, 30: 0.0766224230297619, 31: 0, 32: 0, 33: 0.03939681094736843, 34: 0, 35: 0.12643026693628595, 36: 0.07533659373333336, 37: 0.23267645914592938, 38: 0, 39: 0.08159363443889384, 40: 0.12281122560539214, 41: 0, 42: 0, 43: 0.0847494925857143, 44: 0, 45: 0.1180273175814034, 46: 0, 47: 0.11633839912715122, 48: 0.14877636244242423, 49: 0.036701681703703706, 50: 0.07536150854766735}


['3', '7', '8', '11', '14', '24', '25', '29', '33', '36', '39', '40', '43', '45']
3:	Which is better, Canon or Nikon?
7:	Which is healthiest: coffee, green tea or black tea and why?
8:	What are the advantages and disadvantages of PHP over Python and vice versa?
11:	Should I buy an LCD TV or a plasma TV?
14:	Should one prefer Chinese medicine or Western medicine?
24:	Which one is better, Netflix or Blockbuster?
25:	Which browser is better, Internet Explorer or Firefox?
29:	Which is better, Disneyland or Disney World?
33:	What is better: ASP or PHP?
36:	What IDE is better for Java: NetBeans or Eclipse?
39:	Is Java code more readable than code written in Scala?
40:	Which operating system has better performance: Windows 7 or Windows 8?
43:	Should I prefer a Leica camera over Nikon for portrait photographs?
45:	Which laptop has a better durability: HP or Dell?


# avg score as number of args-labels

In [86]:
arg_labels_each_topic={}
for query_id, targer_resp in topics_argument_labels.items():
    print(len(targer_resp))
    print("==="+str(query_id)+"===")
    percent_labels=0
    for doc in targer_resp.values():
        labels_doc = 0
        len_doc=0
        for sent in doc:
            len_doc = len_doc + len(sent)
            print("======" + str(len(sent))+"=====")
            for token in sent:
                print(token)
                if token['label'].endswith("-B") or token['label'].endswith("-I"):
                    labels_doc = labels_doc + 1
        
        percent_labels_pro_doc = labels_doc/len_doc
        print(labels_doc)
        print(len_doc)
        print(percent_labels_pro_doc)
        percent_labels = percent_labels + percent_labels_pro_doc
                
        print("=======")
    arg_labels_each_topic[query_id] = percent_labels/len(targer_resp) #len(targer_resp) = 15 

15
===1===
======9=====
{'label': 'O', 'prob': '0.7817677', 'token': 'Difference'}
{'label': 'O', 'prob': '0.83270216', 'token': 'Between'}
{'label': 'O', 'prob': '0.802422', 'token': 'Love'}
{'label': 'O', 'prob': '0.8660865', 'token': 'And'}
{'label': 'O', 'prob': '0.83912605', 'token': 'Sex'}
{'label': 'O', 'prob': '0.9082738', 'token': '-'}
{'label': 'O', 'prob': '0.9184673', 'token': 'Student.com'}
{'label': 'O', 'prob': '0.9302571', 'token': 'Articles'}
{'label': 'O', 'prob': '0.89412856', 'token': '.'}
======8=====
{'label': 'O', 'prob': '0.2796378', 'token': 'Sex'}
{'label': 'P-I', 'prob': '0.37010884', 'token': 'may'}
{'label': 'P-I', 'prob': '0.42882606', 'token': 'or'}
{'label': 'P-I', 'prob': '0.45782527', 'token': 'may'}
{'label': 'P-I', 'prob': '0.48444054', 'token': 'not'}
{'label': 'P-I', 'prob': '0.510208', 'token': 'include'}
{'label': 'P-I', 'prob': '0.553338', 'token': 'penetration'}
{'label': 'P-I', 'prob': '0.52148193', 'token': '.'}
======14=====
{'label': 'O', '

{'label': 'O', 'prob': '0.9164257', 'token': ','}
{'label': 'O', 'prob': '0.9158668', 'token': 'Yahoo'}
{'label': 'O', 'prob': '0.907411', 'token': 'or'}
{'label': 'O', 'prob': '0.9025374', 'token': 'Microsoft'}
{'label': 'O', 'prob': '0.894956', 'token': ','}
{'label': 'O', 'prob': '0.8970618', 'token': 'and'}
{'label': 'O', 'prob': '0.89337033', 'token': 'a'}
{'label': 'O', 'prob': '0.89029396', 'token': 'list'}
{'label': 'O', 'prob': '0.8958591', 'token': 'of'}
{'label': 'O', 'prob': '0.8883242', 'token': 'email'}
{'label': 'O', 'prob': '0.8713443', 'token': 'addresses'}
{'label': 'O', 'prob': '0.8761685', 'token': 'are'}
{'label': 'O', 'prob': '0.8622375', 'token': 'passed'}
{'label': 'O', 'prob': '0.86916035', 'token': 'to'}
{'label': 'O', 'prob': '0.853256', 'token': 'us'}
{'label': 'O', 'prob': '0.87263715', 'token': 'via'}
{'label': 'O', 'prob': '0.86289555', 'token': 'secure'}
{'label': 'O', 'prob': '0.85007274', 'token': 'connection'}
{'label': 'O', 'prob': '0.81537175', 'tok

{'label': 'C-I', 'prob': '0.30769572', 'token': 'dishwasher'}
{'label': 'C-I', 'prob': '0.3141759', 'token': 'soap'}
{'label': 'C-I', 'prob': '0.34947953', 'token': 'anymore'}
{'label': 'O', 'prob': '0.2938967', 'token': '.'}
5
51
0.09803921568627451
15
===5===
======17=====
{'label': 'O', 'prob': '0.88923997', 'token': 'Canadas'}
{'label': 'O', 'prob': '0.9126723', 'token': 'best'}
{'label': 'O', 'prob': '0.91331995', 'token': 'city'}
{'label': 'O', 'prob': '0.93472666', 'token': 'to'}
{'label': 'O', 'prob': '0.9180789', 'token': 'live'}
{'label': 'O', 'prob': '0.92522705', 'token': 'in'}
{'label': 'O', 'prob': '0.93000174', 'token': '?'}
{'label': 'O', 'prob': '0.9322191', 'token': '?'}
{'label': 'O', 'prob': '0.9330761', 'token': '?'}
{'label': 'O', 'prob': '0.93253267', 'token': '?'}
{'label': 'O', 'prob': '0.9306582', 'token': '?'}
{'label': 'O', 'prob': '0.9273116', 'token': '?'}
{'label': 'O', 'prob': '0.9219778', 'token': '?'}
{'label': 'O', 'prob': '0.9134674', 'token': '?'}
{

{'label': 'P-I', 'prob': '0.7001644', 'token': 'multiple'}
{'label': 'P-I', 'prob': '0.7066339', 'token': 'paradigms'}
{'label': 'P-I', 'prob': '0.6978766', 'token': 'really'}
{'label': 'P-I', 'prob': '0.6576758', 'token': '.'}
40
53
0.7547169811320755
======11=====
{'label': 'O', 'prob': '0.83871084', 'token': 'John'}
{'label': 'O', 'prob': '0.8214599', 'token': 'Hughes'}
{'label': 'O', 'prob': '0.8470629', 'token': 'on'}
{'label': 'O', 'prob': '0.8575516', 'token': 'Erlang'}
{'label': 'O', 'prob': '0.87144727', 'token': 'and'}
{'label': 'O', 'prob': '0.8852963', 'token': 'Haskell'}
{'label': 'O', 'prob': '0.9230641', 'token': '|'}
{'label': 'O', 'prob': '0.9231353', 'token': 'Lambda'}
{'label': 'O', 'prob': '0.9265774', 'token': 'the'}
{'label': 'O', 'prob': '0.9320738', 'token': 'Ultimate'}
{'label': 'O', 'prob': '0.8973587', 'token': '.'}
======28=====
{'label': 'O', 'prob': '0.82784516', 'token': 'The'}
{'label': 'O', 'prob': '0.8728162', 'token': 'concept'}
{'label': 'O', 'prob':

{'label': 'O', 'prob': '0.5741206', 'token': 'it'}
{'label': 'O', 'prob': '0.6042213', 'token': '’'}
{'label': 'O', 'prob': '0.5798984', 'token': 's'}
{'label': 'O', 'prob': '0.6066545', 'token': 'a'}
{'label': 'O', 'prob': '0.57706803', 'token': 'bit'}
{'label': 'O', 'prob': '0.594794', 'token': 'more'}
{'label': 'O', 'prob': '0.61143637', 'token': 'challenging'}
{'label': 'O', 'prob': '0.64413613', 'token': 'to'}
{'label': 'O', 'prob': '0.6115735', 'token': 'set'}
{'label': 'O', 'prob': '0.62434775', 'token': 'up'}
{'label': 'O', 'prob': '0.62427866', 'token': 'and'}
{'label': 'O', 'prob': '0.6161585', 'token': 'use'}
{'label': 'O', 'prob': '0.6194718', 'token': ','}
{'label': 'O', 'prob': '0.58998775', 'token': 'he'}
{'label': 'O', 'prob': '0.59579986', 'token': 'thinks'}
{'label': 'O', 'prob': '0.5713781', 'token': 'it'}
{'label': 'O', 'prob': '0.5471333', 'token': 'offers'}
{'label': 'O', 'prob': '0.55017346', 'token': 'much'}
{'label': 'O', 'prob': '0.5381516', 'token': 'more'}
{

...

{'label': 'O', 'prob': '0.6595719', 'token': 'For'}
{'label': 'O', 'prob': '0.6899508', 'token': 'routine'}
{'label': 'O', 'prob': '0.6638355', 'token': 'towing'}
{'label': 'O', 'prob': '0.70048505', 'token': 'of'}
{'label': 'O', 'prob': '0.67498237', 'token': 'trailers'}
{'label': 'O', 'prob': '0.6839792', 'token': 'anywhere'}
{'label': 'O', 'prob': '0.6994871', 'token': 'near'}
{'label': 'O', 'prob': '0.6974672', 'token': 'the'}
{'label': 'O', 'prob': '0.67436635', 'token': '10,000'}
{'label': 'O', 'prob': '0.6522571', 'token': 'pounds'}
{'label': 'O', 'prob': '0.66926116', 'token': 'you'}
{'label': 'O', 'prob': '0.67755395', 'token': '&'}
{'label': 'O', 'prob': '0.6849873', 'token': '#'}
{'label': 'O', 'prob': '0.6709853', 'token': 'x27'}
{'label': 'O', 'prob': '0.6985959', 'token': ';'}
{'label': 'O', 'prob': '0.7014197', 'token': 'd'}
{'label': 'O', 'prob': '0.724769', 'token': 'be'}
{'label': 'O', 'prob': '0.6797275', 'token': 'better'}
{'label': 'O', 'prob': '0.6522051', 'token'

{'label': 'O', 'prob': '0.7695351', 'token': 'stock'}
{'label': 'O', 'prob': '0.7287528', 'token': 'price'}
{'label': 'O', 'prob': '0.70796156', 'token': '.'}
0
56
0.0
======9=====
{'label': 'O', 'prob': '0.8553911', 'token': 'Apple'}
{'label': 'O', 'prob': '0.8091668', 'token': 'introduces'}
{'label': 'O', 'prob': '0.83586997', 'token': 'dividend'}
{'label': 'O', 'prob': '0.8294873', 'token': ','}
{'label': 'O', 'prob': '0.8297059', 'token': 'share'}
{'label': 'O', 'prob': '0.84298784', 'token': 'buybacks'}
{'label': 'O', 'prob': '0.90669686', 'token': '–'}
{'label': 'O', 'prob': '0.9308685', 'token': 'USATODAY.com'}
{'label': 'O', 'prob': '0.88042295', 'token': '.'}
======27=====
{'label': 'O', 'prob': '0.8112336', 'token': 'The'}
{'label': 'O', 'prob': '0.8393613', 'token': 'buybacks'}
{'label': 'O', 'prob': '0.8572218', 'token': 'by'}
{'label': 'O', 'prob': '0.8789036', 'token': 'Apple'}
{'label': 'O', 'prob': '0.8706765', 'token': ','}
{'label': 'O', 'prob': '0.86910075', 'token':

{'label': 'O', 'prob': '0.6810193', 'token': ','}
{'label': 'O', 'prob': '0.6853103', 'token': 'a'}
{'label': 'O', 'prob': '0.6552508', 'token': 'darker'}
{'label': 'O', 'prob': '0.66719764', 'token': ','}
{'label': 'O', 'prob': '0.65212053', 'token': 'more'}
{'label': 'O', 'prob': '0.6400047', 'token': 'flavorful'}
{'label': 'O', 'prob': '0.6514495', 'token': 'beer'}
{'label': 'O', 'prob': '0.635327', 'token': ','}
{'label': 'O', 'prob': '0.6071839', 'token': 'was'}
{'label': 'O', 'prob': '0.63625187', 'token': 'added'}
{'label': 'O', 'prob': '0.6703427', 'token': 'to'}
{'label': 'O', 'prob': '0.67175424', 'token': 'this'}
{'label': 'O', 'prob': '0.63299704', 'token': 'sub-line'}
{'label': 'O', 'prob': '0.5939133', 'token': '.'}
======14=====
{'label': 'O', 'prob': '0.68761027', 'token': 'Essentially'}
{'label': 'O', 'prob': '0.77909577', 'token': 'the'}
{'label': 'O', 'prob': '0.72074246', 'token': 'first'}
{'label': 'O', 'prob': '0.72611487', 'token': 'mainstream'}
{'label': 'O', 'p

{'label': 'O', 'prob': '0.96058947', 'token': 'You'}
{'label': 'O', 'prob': '0.9677203', 'token': 'Think'}
{'label': 'O', 'prob': '0.96689266', 'token': '«'}
{'label': 'O', 'prob': '0.96029294', 'token': 'More'}
{'label': 'O', 'prob': '0.95713854', 'token': 'Thyme'}
{'label': 'O', 'prob': '0.9442151', 'token': 'Than'}
{'label': 'O', 'prob': '0.9249353', 'token': 'Dough'}
{'label': 'O', 'prob': '0.8834864', 'token': '.'}
======8=====
{'label': 'O', 'prob': '0.93119204', 'token': 'Selling'}
{'label': 'O', 'prob': '0.98596954', 'token': 'BooksFoods'}
{'label': 'O', 'prob': '0.9850841', 'token': 'That'}
{'label': 'O', 'prob': '0.98717', 'token': 'Are'}
{'label': 'O', 'prob': '0.9806354', 'token': 'Healthier'}
{'label': 'O', 'prob': '0.97856814', 'token': 'Than'}
{'label': 'O', 'prob': '0.97628266', 'token': 'You'}
{'label': 'O', 'prob': '0.95704955', 'token': 'Think'}
0
21
0.0
======13=====
{'label': 'O', 'prob': '0.5589699', 'token': 'Congress'}
{'label': 'O', 'prob': '0.64437395', 'token

{'label': 'O', 'prob': '0.60426885', 'token': 'Is'}
{'label': 'O', 'prob': '0.5274536', 'token': 'More'}
{'label': 'B-I', 'prob': '0.49857903', 'token': 'Influential'}
{'label': 'B-I', 'prob': '0.48587266', 'token': '--'}
{'label': 'B-I', 'prob': '0.49052998', 'token': 'Jesse'}
{'label': 'B-I', 'prob': '0.5018206', 'token': 'Owens'}
{'label': 'B-I', 'prob': '0.4722836', 'token': 'or'}
{'label': 'B-I', 'prob': '0.5084266', 'token': 'Dwight'}
{'label': 'B-I', 'prob': '0.4890997', 'token': 'Eisenhower'}
{'label': 'O', 'prob': '0.49506575', 'token': '?'}
{'label': 'B-I', 'prob': '0.48086172', 'token': '.'}
======11=====
{'label': 'O', 'prob': '0.6533959', 'token': 'Who'}
{'label': 'O', 'prob': '0.63389534', 'token': 'Is'}
{'label': 'O', 'prob': '0.55984664', 'token': 'More'}
{'label': 'B-I', 'prob': '0.46895832', 'token': 'Influential'}
{'label': 'O', 'prob': '0.4620618', 'token': '--'}
{'label': 'O', 'prob': '0.46813023', 'token': 'Jesse'}
{'label': 'B-I', 'prob': '0.48029688', 'token': '

In [88]:
arg_labels_each_topic

{1: 0.04135481175329839,
 2: 0.2638870674866397,
 3: 0.04776378221539512,
 4: 0.06765854937315233,
 5: 0.04615707433429937,
 7: 0.0848443239359041,
 8: 0.22447967097186006,
 9: 0.06025290923546737,
 10: 0.250571666285147,
 11: 0.08175881623250043,
 12: 0.20073258157431864,
 14: 0.3741880683626785,
 15: 0.07895360112380424,
 16: 0.019438078261607674,
 17: 0.11748580133396688,
 18: 0.017316561844863732,
 19: 0.016938967136150233,
 20: 0.04301124606387765,
 21: 0.10183601609657947,
 22: 0.03018867924528302,
 23: 0.017104377104377105,
 24: 0.014084507042253521,
 25: 0.11866688819391479,
 26: 0.029071697344454032,
 27: 0.10219331582826609,
 28: 0.016930137684854667,
 29: 0.09132248644138506,
 30: 0.051927494119274956,
 31: 0.20347766023827146,
 32: 0.0,
 33: 0.06789639614711132,
 34: 0.023399810066476734,
 35: 0.19727238759496826,
 36: 0.10899594599515156,
 37: 0.2910853099541625,
 38: 0.03159961612962463,
 39: 0.12295637575158504,
 40: 0.22674077278218255,
 41: 0.30463963963963964,
 42: 0.

In [87]:
arg_avgs_each_query=arg_labels_each_topic
plot_comparing()

['3', '7', '8', '11', '14', '24', '25', '29', '33', '36', '39', '40', '43', '45']
3:	Which is better, Canon or Nikon?
7:	Which is healthiest: coffee, green tea or black tea and why?
8:	What are the advantages and disadvantages of PHP over Python and vice versa?
11:	Should I buy an LCD TV or a plasma TV?
14:	Should one prefer Chinese medicine or Western medicine?
24:	Which one is better, Netflix or Blockbuster?
25:	Which browser is better, Internet Explorer or Firefox?
29:	Which is better, Disneyland or Disney World?
33:	What is better: ASP or PHP?
36:	What IDE is better for Java: NetBeans or Eclipse?
39:	Is Java code more readable than code written in Scala?
40:	Which operating system has better performance: Windows 7 or Windows 8?
43:	Should I prefer a Leica camera over Nikon for portrait photographs?
45:	Which laptop has a better durability: HP or Dell?


# comparing two targer models

In [13]:
#targer_model="classifyWD_dep"
for targer_model in ["classifyWD_dep", "classifyWD"]:
    underscore=0.0
    while underscore<=0.7:
        avg_scores = get_results(targer_model, underscore)
        underscore=round(underscore+0.05,2)

Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argument query
{}
Not argume

In [14]:
#{'targer_modell': 
 #   [{0:nDCG}, {0.1: nDCG},...]
nDCGs_by_model={}
scores_by_model={}
for targer_model in ["classifyWD_dep", "classifyWD"]:
    scores=[]
    nDCGs=[]
    
    underscore=0.0
    while underscore<=0.7:
        scores.append(underscore)
        score_string=score_string=str(underscore).replace('.','')
        nDCGs.append(nDCG(targer_model, score_string))
        underscore=round(underscore+0.05,2)
    nDCGs_by_model[targer_model] = nDCGs
    scores_by_model[targer_model] = scores
df= pd.DataFrame()
df['score']= scores
#df['ndcg']= [e[1]for e in list_weights]
df['nDCG5'] = nDCGs_by_model['classifyWD_dep']
df=df.sort_values('nDCG5', ascending=False)
df

TypeError: '<' not supported between instances of 'dict' and 'dict'